In [6]:
print("hello world")

hello world


In [7]:
import cv2
import time
from nescore import RetroEmulator
from key import KeyboardController

core_path = "E:/uup/nes/pytest/fceumm_libretro.dll" 
rom_path = "E:/uup/nes/pytest/super-mario-bros.nes"  

def main():
    # 创建模拟器实例
    emu = RetroEmulator(core_path, rom_path)
    
    time.sleep(0.01)
    # 初始化模拟器
    if not emu.init_core():
        print("初始化模拟器核心失败")
        return
    
    # 加载游戏ROM
    if not emu.load_game():
        print("加载游戏ROM失败")
        return
    
    # 创建键盘控制器
    keyboard_controller = KeyboardController(emu)
    
    # 启动键盘监听
    keyboard_controller.start()
    
    # 创建显示窗口
    cv2.namedWindow('NES Emulator', cv2.WINDOW_NORMAL)
    
    frame_count = 0
    
    print("模拟器已启动，使用方向键和Z/X/Enter/Shift键控制")
    print("按ESC键退出模拟器")
    
    # 示例：外部输入控制（AI输出的状态通过这个来传递到模拟器）
    example_external_input = {
        'UP': False, 'DOWN': False, 'LEFT': False, 'RIGHT': False,
        'A': False, 'B': False, 'START': False, 'SELECT': False,
    }
    
    try:
        # 主循环
        while keyboard_controller.running:
            # 运行一帧并获取图片
            frame = emu.run()
            
            # 显示视频帧
            if frame is not None:
                cv2.imshow('NES Emulator', frame)
                frame_count += 1
                
                # 定期保存图片,覆盖之前的# 每100帧保存一张图片
                if frame_count % 100 == 0:
                    filename = f"output.jpg"
                    cv2.imwrite(filename, frame)
            
            # 处理OpenCV窗口事件（保持窗口响应）
            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC键
                break
            
            # 示例：模拟外部输入控制（每100帧触发一次右方向键）
            if frame_count % 100 == 50:  # 在第50帧触发
                example_external_input['START'] = True
                example_external_input['A'] = True
                result = keyboard_controller.update_state_from_external(example_external_input)
                #print("外部输入: 右方向键 + A键")
            elif frame_count % 100 == 51:  # 在第51帧释放
                example_external_input['START'] = False
                example_external_input['A'] = False
                result = keyboard_controller.update_state_from_external(example_external_input)
                #print("外部输入: 释放所有按键")
            
            # 显示当前控制器状态（可选）
            if frame_count % 100 == 0:
                current_state = keyboard_controller.get_current_state()
                #print(f"当前控制器状态: {current_state}")
            
            # 稍微延迟以控制帧率
            time.sleep(0.01)
    
    except KeyboardInterrupt:
        print("程序被中断")
    
    finally:
        # 清理资源
        keyboard_controller.stop()
        cv2.destroyAllWindows()
        emu.shutdown()
        print("模拟器已关闭")


if __name__ == "__main__":
    main()

键盘监听已启动，按ESC键退出
模拟器已启动，使用方向键和Z/X/Enter/Shift键控制
按ESC键退出模拟器
键盘监听已停止
模拟器已关闭
